# Scrapping MILI

## Este código permite que você pesquise e extraia dados de carros do mercado livre podendo ser generalizado

importando os pacotes:

In [ ]:
import pandas as pd 
import numpy as np
from selenium import webdriver
from requests import get
from selenium.webdriver.common.keys import Keys
import time

Aqui eu defini o driver:

In [ ]:
driver = webdriver.Chrome(executable_path='./chr_drv/chromedriver.exe')

### Parte_1_: Acessando a página

In [ ]:
url = "https://lista.mercadolivre.com.br/veiculos/carros-caminhonetes/#deal_print_id=9f973e90-e73e-11eb-b5cc-d5f53f9f352d&c_id=carousel&c_element_order=1&c_campaign=CARROS-E-CAMINHONETES&c_uid=9f973e90-e73e-11eb-b5cc-d5f53f9f352d"
pagina = driver.get(url)

## Definindo um critério:

### <span style="color:green">Observação: esse time é para que nosso boot espere um intervalo de 3 segundos para fazer a operação</span>

In [4]:
time.sleep(3)
valor = "45000"
caixadevalor = driver.find_element_by_css_selector('[data-testid="Maximum-price"]')
caixadevalor.clear()
caixadevalor.send_keys(valor)

## Aqui defini o botão para aplicar nosso critério
### <span style="color:red">Observação: aqui defini um botão para acessar o cookie do site e clicar também seria nosso find_by_id</span>

In [5]:
driver.find_element_by_css_selector('[data-testid="submit-price"]').click()
time.sleep(2)
driver.find_element_by_id("newCookieDisclaimerButton").click()

## Definição das listas e do contador posteriormente usados:

In [6]:
carro = []
vale = []
links = []
count = 1


#### Aqui eu defini quantas paginas iremos acessar com o n
#### Esse v será nosso time que poderá ser no maximo igual a 5 segundos, caso contrário ele reseta

In [ ]:
n = 3
v = n
if n>5:
    v=5

## - Pulo do gato:
### o primeiro for passa a página
### o if impede que ele pule mais páginas que o necessário(talvez dê pra ganhar aqui fazendo n-1)
### o segundo, terceiro e quarto for preenche a lista com o link, descrição geral e preço

In [7]:




for j in range(n):
    time.sleep(v)
    
    carros= driver.find_elements_by_xpath('/html/body/main/div/div/section/ol[*]/li[*]/div/div/a/div/div[*]/h2')
   
    pula_pag = driver.find_element_by_css_selector('[title="Seguinte"]')
    preço= driver.find_elements_by_xpath("/html/body/main/div/div/section/ol[*]/li[*]/div/div/a/div/div[*]/div/div/span/span[2]/span[2]")
    
    list_links = driver.find_elements_by_xpath('/html/body/main/div/div/section/ol[*]/li[*]/div/div/a')

    for i in list_links:
        links.append( i.get_attribute('href'))
    for k in carros:
        carro.append(k.text)
            
    for q in preço:
       
        vale.append(q.text)
    
        
    if count<n:
        count+=1
        pula_pag.click()
       


## Cria o DF, Organiza e Salva em CSV

In [8]:
carr = pd.DataFrame(carro)
price = pd.DataFrame(vale).astype("float")
url =pd.DataFrame(links)



In [9]:
pd.get_option('max_colwidth')

50

In [10]:
pd.set_option('max_colwidth',
              None)

In [18]:
carr.rename({0:"Nome"}, 
            axis=1,
            inplace=True)

price.rename({0:"Valor em milhar"}
             ,axis=1, 
             inplace=True)

url.rename({0:"URL"},
           axis=1,
           inplace = True)


In [12]:
data = pd.concat([carr,price,url],axis=1)

In [17]:
data2 = data.sort_values('Valor em milhar',ascending=True )
data2.reset_index(drop=True, inplace=True)

In [14]:
data2.to_csv('Carros.csv',sep=',', encoding = 'utf-8-sig')